# Regression Model on Merged Dataset

In [1]:
# all imports
import pandas as pd
import os
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from math import sqrt
import statsmodels.formula.api as smf
import statsmodels.stats as stats

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import statsmodels.api as sm

In [2]:
# cwdir = os.getcwd()
# air = pd.read_csv(cwdir + "\\Final Airfar Data for Midterm.csv")

In [3]:
# car = pd.read_csv(cwdir + "\\Final Car Rental Data for Midterm.csv")
#car = pd.read_csv(cwdir + "\\car_withLOR.csv")

In [4]:
# air.head()

In [5]:
# car.head()

In [6]:
# air_lax = air[air["TO_AIRPORT"] == "LAX"]
# air_lax.head()

In [7]:
# car_lax = car[car["LOCATION"] == "LAX"]
# car_lax.head()

In [8]:
# merged = pd.merge(car_lax, air_lax,  how='left', left_on=['WEEK_NO','DAYOFWEEK(PDATE)'], right_on = ['EXTRACT(WEEK FROM DDATE)','DAYOFWEEK(DDATE)'])

In [9]:
# merged

In [10]:
# merged.to_csv("Merged Data.csv")

In [11]:
# run from here next time

In [12]:
merged = pd.read_csv("../../Data/Merged Data.csv")

In [13]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163416 entries, 0 to 163415
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                163416 non-null  int64  
 1   LOCATION                  163416 non-null  object 
 2   AGENCY                    163416 non-null  object 
 3   WEEK_NO                   163416 non-null  int64  
 4   DAYOFWEEK(PDATE)          163416 non-null  int64  
 5   AVG(DAILY_FARE)           163416 non-null  float64
 6   MIN(DAILY_FARE)           163416 non-null  float64
 7   MAX(DAILY_FARE)           163416 non-null  float64
 8   MEDIAN(DAILY_FARE)        163416 non-null  float64
 9   STDDEV(DAILY_FARE)        163416 non-null  float64
 10  NO_RECORDS_x              163416 non-null  int64  
 11  TO_AIRPORT                163416 non-null  object 
 12  CXR                       163416 non-null  object 
 13  FROM_AIRPORT              163416 non-null  o

In [14]:
merged.head()

Unnamed: 0 LOCATION AGENCY  WEEK_NO  DAYOFWEEK(PDATE)  AVG(DAILY_FARE)  \
0           0      LAX     AL        1                 0        82.560914   
1           1      LAX     AL        1                 0        82.560914   
2           2      LAX     AL        1                 0        82.560914   
3           3      LAX     AL        1                 0        82.560914   
4           4      LAX     AL        1                 0        82.560914   

   MIN(DAILY_FARE)  MAX(DAILY_FARE)  MEDIAN(DAILY_FARE)  STDDEV(DAILY_FARE)  \
0        20.521111          342.405           79.508889           38.522142   
1        20.521111          342.405           79.508889           38.522142   
2        20.521111          342.405           79.508889           38.522142   
3        20.521111          342.405           79.508889           38.522142   
4        20.521111          342.405           79.508889           38.522142   

   ...  FROM_AIRPORT EXTRACT(WEEK FROM DDATE) DAYOFWEEK(DDATE)   AVG(FARE)  \
0  ...           JFK                        1                0  278.100000   
1  ...           OGG                        1                0  619.000000   
2  ...           ORD                        1                0  245.500000   
3  ...           PDX                        1                0   94.000000   
4  ...           SEA                        1                0  251.229167   

   MIN(FARE)  MAX(FARE)  MEDIAN(FARE)  STDDEV(FARE)  NO_RECORDS_y  DISTANCE  
0      243.0      429.0         243.0     66.108413            10      2469  
1      589.0      649.0         619.0     42.426407             2      2483  
2      178.0      313.0         245.5     95.459415             2      1741  
3       69.0      119.0          99.0     15.063966            14       835  
4       89.0      599.0         219.5    152.098266            48       955  

[5 rows x 23 columns]

In [15]:
weekdays = [1,2,3,4]
merged["weekend"] = np.where(merged.iloc[:,4].isin(weekdays) == True, 0, 1)

In [16]:
merged_reg = merged.copy()

#Dropped FROM_AIRPORT , DISTANCE , CXR
#Added No_records_y ,  AVG(FARE) into regression --this will include Airfare as a feature for Care Fare prediction
drop_columns = ["FROM_AIRPORT", "DISTANCE", "CXR","DAYOFWEEK(PDATE)","Unnamed: 0", "LOCATION", "MIN(DAILY_FARE)", "MAX(DAILY_FARE)", "MEDIAN(DAILY_FARE)","STDDEV(DAILY_FARE)","TO_AIRPORT", "EXTRACT(WEEK FROM DDATE)", "DAYOFWEEK(DDATE)", "MIN(FARE)","MAX(FARE)", "MEDIAN(FARE)", "STDDEV(FARE)"]
merged_reg.drop(columns = drop_columns , axis=1, inplace = True)


"""
#Group by everything except Label column
merged_regression_group = merged_reg.groupby(list(merged_regression.columns[:-1]) , as_index=False)
merged_regression_group.aggregate(np.mean)

merged_regression = pd.DataFrame(merged_regression_group)"""
merged_regression = pd.get_dummies(merged_reg,columns=['AGENCY' , "WEEK_NO"], drop_first=True)
merged_regression.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163416 entries, 0 to 163415
Data columns (total 63 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   AVG(DAILY_FARE)  163416 non-null  float64
 1   NO_RECORDS_x     163416 non-null  int64  
 2   AVG(FARE)        163416 non-null  float64
 3   NO_RECORDS_y     163416 non-null  int64  
 4   weekend          163416 non-null  int64  
 5   AGENCY_ET        163416 non-null  uint8  
 6   AGENCY_ZD        163416 non-null  uint8  
 7   AGENCY_ZE        163416 non-null  uint8  
 8   AGENCY_ZI        163416 non-null  uint8  
 9   AGENCY_ZL        163416 non-null  uint8  
 10  AGENCY_ZR        163416 non-null  uint8  
 11  AGENCY_ZT        163416 non-null  uint8  
 12  WEEK_NO_2        163416 non-null  uint8  
 13  WEEK_NO_3        163416 non-null  uint8  
 14  WEEK_NO_4        163416 non-null  uint8  
 15  WEEK_NO_5        163416 non-null  uint8  
 16  WEEK_NO_6        163416 non-null  uint

In [17]:
#Rearranging columns
merged_regression = merged_regression[[col for col in merged_regression.columns if col not in ['AVG(DAILY_FARE)']] + [ 'AVG(DAILY_FARE)']]


#Removing paranthesis from the column names
regression_columns = [ col.replace('(' , '_').replace(')' , '') for col in merged_regression.columns ]
merged_regression.columns = regression_columns

merged_regression.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163416 entries, 0 to 163415
Data columns (total 63 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   NO_RECORDS_x    163416 non-null  int64  
 1   AVG_FARE        163416 non-null  float64
 2   NO_RECORDS_y    163416 non-null  int64  
 3   weekend         163416 non-null  int64  
 4   AGENCY_ET       163416 non-null  uint8  
 5   AGENCY_ZD       163416 non-null  uint8  
 6   AGENCY_ZE       163416 non-null  uint8  
 7   AGENCY_ZI       163416 non-null  uint8  
 8   AGENCY_ZL       163416 non-null  uint8  
 9   AGENCY_ZR       163416 non-null  uint8  
 10  AGENCY_ZT       163416 non-null  uint8  
 11  WEEK_NO_2       163416 non-null  uint8  
 12  WEEK_NO_3       163416 non-null  uint8  
 13  WEEK_NO_4       163416 non-null  uint8  
 14  WEEK_NO_5       163416 non-null  uint8  
 15  WEEK_NO_6       163416 non-null  uint8  
 16  WEEK_NO_7       163416 non-null  uint8  
 17  WEEK_NO_8 

In [18]:
# X = merged_regression.iloc[: , :-1]
# print("X" , X.shape)
# y = merged_regression.iloc[:, -1]
# print("y" , y.shape)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X (163416, 62)
y (163416,)


In [19]:
merged_regression.shape

(163416, 63)

In [20]:
def decode(datum):
    #print(datum)
    value = np.argmax(datum)
    #print("value",value)
    value = int(value[8:])
    #print("int value",value)
    
    if value == 2:
        #print("datum",int(datum[0]))
        if(int(datum[0]))==0:
            value=1
        else:
            value=2         
    
    return value

#a = [0,0,0,0]
#decode(a)
#decode(merged_regression.iloc[5000 , 11:-1])

In [21]:
decode(merged_regression.iloc[5000 , 11:-1])

IndexError: invalid index to scalar variable.

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.stats as stats

In [ ]:
x_features = regression_columns[0]
for xm in regression_columns[1:-1]:
    x_features = x_features+'+'+xm
x_features

In [ ]:
regression_columns[-1] + "~"+ x_features

In [ ]:
merged_regression.shape

In [ ]:
reg_ols = smf.ols(formula= regression_columns[-1] + "~"+ x_features  , data = merged_regression ).fit()

In [ ]:
reg_ols.summary()

In [ ]:
result_param = abs(reg_ols.params)
result_param.sort_values()

In [ ]:
fig = plt.figure(figsize=(18,5))
plt.plot(result_param)
plt.xticks(rotation = 75)
plt.legend(["Coefficient values for features"])
plt.show()

In [ ]:
fig = plt.figure(figsize = (20,7))

#axes = fig.add_axes([0.1, 0.01, 0.5 ,0.8])
#plt.scatter(reg_ols.pvalues.index , reg_ols.pvalues)
plt.plot(reg_ols.pvalues)
plt.xticks(rotation = 75)
plt.yticks(np.arange(0, 1,0.1))
plt.legend(["p value of variables"])
plt.show()

In [ ]:
plt.hist(reg_ols.pvalues)

In [ ]:
reg_ols.pvalues[reg_ols.pvalues>0.05]

In [ ]:
#NO_RECORDS_y    0.418647 -
#CXR_HA          0.520692 -
#WEEK_NO_3       0.064827 - Jan 14- Jan 28
#WEEK_NO_17      0.898010 - Apr 22 - Apr 28
#WEEK_NO_43      0.998704 - Oct 21 - Oct 27
#WEEK_NO_45      0.94534  - Nov 4 - NOv 10

In [ ]:
fig = plt.figure(figsize = (20,7))

#axes = fig.add_axes([0.1, 0.01, 0.5 ,0.8])
#plt.scatter(reg_ols.pvalues.index , reg_ols.pvalues)
plt.plot(reg_ols.pvalues)
plt.xticks(rotation = 75)
plt.yticks(np.arange(0, 1,0.1))
plt.legend(["p value of variables"])
plt.show()

In [ ]:
y_pred = reg_ols.predict(merged_regression.iloc[: , :-1])

In [ ]:
y_pred.shape

In [ ]:
merged_regression['y_pred'] = [ yhat for yhat in y_pred]
merged_regression.iloc[:5,-2:]

In [ ]:
merged_regression.iloc[2 , 11:]

In [ ]:
merged_regression.shape

In [ ]:
#merged_regression.iloc[10000 , 11:-3]

In [ ]:
merged_regression['decode_week'] = 0
#merged_regression.shape[0]
for i in range(1000):
    merged_regression.iloc[i,-1] = decode(merged_regression.iloc[i , 11:-3])

In [ ]:
merged_regression.iloc[999, -1]

In [ ]:
for i in range(1000,50000,1):
    merged_regression.iloc[i,-1] = decode(merged_regression.iloc[i , 11:-3])
merged_regression.iloc[9999,-1]

In [ ]:
for i in range(50000,100000,1):
    merged_regression.iloc[i,-1] = decode(merged_regression.iloc[i , 11:-3])
merged_regression.iloc[99999,-1]

In [ ]:
for i in range(100000,merged_regression.shape[0],1):
    merged_regression.iloc[i,-1] = decode(merged_regression.iloc[i , 11:-3])
merged_regression.iloc[-1,-1]

In [ ]:
result_merged_reg = merged_regression.copy()

In [ ]:
result_merged_reg.drop(columns = result_merged_reg.columns[11:-3], inplace=True)
result_merged_reg.columns

In [ ]:
result_merged_reg.groupby(by = ["decode_week"])["y_pred"].min()

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(result_merged_reg.groupby(by = ["decode_week"])["y_pred"].min())
plt.plot(result_merged_reg.groupby(by = ["decode_week"])["AVG_DAILY_FARE"].min(), color="green")
plt.xticks(range(53))
plt.grid()
#plt.plot( y=result_merged_reg['AVG_DAILY_FARE'])
#plt.plot(merged_regression['y_pred'].rolling(window = 30) , color = 'red')
plt.show()

In [ ]:
#withWeekIndex = pd.merge(y_test, y_pred,  how='left', left_on=index, right_on = index)
#y_test['y_pred']= [pred for pred in y_pred]
#y_test[:10]

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(merged.groupby(by = ["WEEK_NO"])["AVG(DAILY_FARE)"].mean())
plt.plot(merged.groupby(by = ["WEEK_NO"])["AVG(DAILY_FARE)"].mean().rolling(window=3).mean(), 'k-', color="red")
plt.xticks(range(0,53))
plt.grid()
plt.legend(["mean price per mile", "rolling average 3 weeks"])
plt.show()

In [ ]:
print("Complete data",merged_regression.shape)
X = merged_regression.iloc[:, :-1]
print("X features" , X.shape)
y = merged_regression["AVG_FARE"]
print( "Y label" , y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Code below not used. Just for ref

In [ ]:


reg = LinearRegression()
reg.fit(X_train, y_train)
print("Train RMSE error", sqrt(mean_squared_error(y_train, reg.predict(X_train))))
print("Test RMSE error", sqrt(mean_squared_error(y_test, reg.predict(X_test))))
print("Coefficients")
print("Intercept" , reg.intercept_)
for x, y in zip(X_train.columns.tolist(), reg.coef_):
    print(x,round(y,2))

In [ ]:
clf = linear_model.Lasso()
clf.fit(X_train, y_train)
print("Train RMSE error", sqrt(mean_squared_error(y_train, clf.predict(X_train))))
print("Test RMSE error", sqrt(mean_squared_error(y_test, clf.predict(X_test))))
print("Coefficients")
for x, y in zip(X_train.columns.tolist(), clf.coef_):
    print(x,round(y,2))